# Margins for NSE
-[x] Build / Extract the chains    
-[ ] Get undPrices   
-[ ] Integrate undPrices to the chains   

-[ ] Get the lots   
-[ ] Get margins for with qty as the lot   
-[ ] Make logic for margins from NSE rules   
-[ ] For margins with NaN, replace margins with the one derived from logic   
-[ ] Integrate margins to chains

In [3]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [4]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER
COLS = _vars.COLS[0]
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

In [5]:
# Build chains
from utils import make_chains_with_margins
df_chains = make_chains_with_margins(MARKET=MARKET)

2024-03-24 18:37:30.013 | ERROR    | utils:get_pickle:530 - file not found: C:\Users\kashi\python\options\data\nse\unds.pkl
2024-03-24 18:37:30.018 | INFO     | utils:get_file_age:399 - C:\Users\kashi\python\options\data\nse\unds.pkl file is not found
Qualifying NSE Unds: 100%|██████████| 181/181 [00:04<00:00, 39.52it/s]
2024-03-24 18:37:42.519 | INFO     | utils:get_file_age:399 - C:\Users\kashi\python\options\data\nse\df_chains.pkl file is not found
Qualifying NSE Unds: 100%|██████████| 181/181 [00:04<00:00, 37.90it/s]
2024-03-24 18:40:16.835 | INFO     | utils:get_file_age:399 - C:\Users\kashi\python\options\data\nse\df_und_prices.pkl file is not found
NSE margins second run:100%|████████████████████| 98/98 [00:04<00:00, 23.65it/s]


# TODO
[x] Make a function for expPrice for nakeds
 - Save nakeds for sow
 - Fix sow program
 - Fix choices

In [6]:
# Build / Get the chains
df_chains = pd.read_pickle(DATAPATH / 'df_chains.pkl')
df_chains.head()

,symbol,expiry,strike,right,undId,dte,multiplier,localSymbol,undPrice,iv,sigma,strike_sdev,exchange,lot,margin,comm
0,BANKNIFTY,20240327,27000.0,P,56994300,2.972239,1,BANKNIFTY,46685.0,0.1455,612.96359,-32.114469,NSE,15.0,8010.51,1.333333
1,BANKNIFTY,20240327,28500.0,P,56994300,2.972239,1,BANKNIFTY,46685.0,0.1455,612.96359,-29.667341,NSE,15.0,8010.51,1.333333
2,BANKNIFTY,20240327,30000.0,P,56994300,2.972239,1,BANKNIFTY,46685.0,0.1455,612.96359,-27.220214,NSE,15.0,8010.51,1.333333
3,BANKNIFTY,20240327,30500.0,P,56994300,2.972239,1,BANKNIFTY,46685.0,0.1455,612.96359,-26.404505,NSE,15.0,8010.51,1.333333
4,BANKNIFTY,20240327,31000.0,P,56994300,2.972239,1,BANKNIFTY,46685.0,0.1455,612.96359,-25.588796,NSE,15.0,8010.51,1.333333


In [7]:
# Make raw targets from the chains
from utils import raw_target_opts
df_ch = raw_target_opts(df_chains)
df_ch.head()

,symbol,expiry,strike,right,undId,dte,multiplier,localSymbol,undPrice,iv,sigma,strike_sdev,exchange,lot,margin,comm
5236,AARTIIND,20240328,585.0,P,56988365,3.972235,1,AARTIIND,648.5,0.322032,21.786103,-2.914702,NSE,1000.0,147.74847,0.02
5237,AARTIIND,20240328,590.0,P,56988365,3.972235,1,AARTIIND,648.5,0.322032,21.786103,-2.685198,NSE,1000.0,147.74847,0.02
5264,AARTIIND,20240328,725.0,C,56988365,3.972235,1,AARTIIND,648.5,0.322032,21.786103,3.511413,NSE,1000.0,147.74847,0.02
5265,AARTIIND,20240328,730.0,C,56988365,3.972235,1,AARTIIND,648.5,0.322032,21.786103,3.740917,NSE,1000.0,147.74847,0.02
5325,AARTIIND,20240425,495.0,P,56988365,31.972235,1,AARTIIND,648.5,0.322032,61.808554,-2.483475,NSE,1000.0,147.74847,0.02


In [8]:
# qualify the targets
from utils import qualify_naked_opts
df_naked_targets = qualify_naked_opts(df_ch)

NSE naked opt margins100%|██████████████████| 1856/1856 [02:01<00:00, 15.29it/s]


In [18]:
import asyncio

import numpy as np

from utils import get_pickle, get_prec, get_prices_with_ivs, pickle_me, get_market_name

pickle_me(df_naked_targets, DATAPATH / 'df_naked_targets.pkl')

df = get_pickle(DATAPATH / 'df_naked_targets.pkl')

In [19]:
def nakeds_with_expPrice(df: pd.DataFrame) -> pd.DataFrame:
    """Makes a df with expected price"""

    MARKET = get_market_name(df)
    _vars = Vars(MARKET)

    MINEXPROM = _vars.MINEXPROM
    PREC = _vars.PREC
    MINOPTSELLPRICE = _vars.MINOPTSELLPRICE

    # rename df iv
    df.rename(columns={'iv':'und_iv'}, inplace=True, errors='ignore')

    # get prices of nakeds
    df_prices = asyncio.run(get_prices_with_ivs(port=PORT, input_contracts=df.contract))

    # convert expiry to string to combine with df
    dfp = df_prices.assign(expiry = df_prices.expiry.dt.strftime("%Y%m%d"))
    dfp[COLS + ['optPrice']]

    # integrate option price
    df_nakeds = pd.merge(df, dfp[COLS + ['optPrice']], on=COLS)

    # make zero margin as nan
    zero_margin_condition = df_nakeds.margin == 0
    df_nakeds.loc[zero_margin_condition, 'margin'] = np.nan

    # remove nans from margins
    df_nakeds = df_nakeds[~df_nakeds.margin.isnull()].reset_index(drop=True)

    # fill missing commissions with max per symbol
    commissions = df_nakeds.groupby('symbol').comm.max().to_dict()
    df_nakeds.comm = df_nakeds.comm.fillna(df_nakeds.symbol.map(commissions))

    # fill remaining commissions
    df_nakeds.comm = df_nakeds.comm.fillna(max(commissions.values()))

    # fill margins
    mgn_dict = df_nakeds.groupby('symbol').margin.max().to_dict()
    cond = df_nakeds.margin.isnull()
    df_nakeds.loc[cond, 'margin'] = df_nakeds[cond].symbol.map(mgn_dict)

    # Get precise expected prices
    xp = ((MINEXPROM*df_nakeds.dte/365*df_nakeds.margin) +
            df_nakeds.comm)/df_nakeds.lot/df_nakeds.multiplier

    # Set the minimum option selling price
    xp[xp < MINOPTSELLPRICE] = MINOPTSELLPRICE

    # Make the expected Price
    expPrice = pd.concat([xp, 
                            df_nakeds.optPrice], axis=1)\
                            .max(axis=1)
    expPrice = expPrice.apply(lambda x: get_prec(x, PREC))
    dfn = df_nakeds.assign(expPrice = expPrice)

    # Limit the number of options to trade
    cond = dfn.expPrice < 3* dfn.optPrice
    dfn = dfn[cond]

    return dfn


In [20]:
dfn = nakeds_with_expPrice(df)

Getting ticker prices100%|██████████████████| 1856/1856 [09:51<00:00,  3.14it/s]


In [ ]:
# GET PRICE AND IV
df.rename(columns={'iv':'und_iv'}, inplace=True, errors='ignore')

In [ ]:
df_prices = asyncio.run(get_prices_with_ivs(port=PORT, input_contracts=df.contract))


In [ ]:
# convert expiry to string to combine with df
dfp = df_prices.assign(expiry = df_prices.expiry.dt.strftime("%Y%m%d"))
dfp[COLS + ['optPrice']]

In [ ]:
df_nakeds = pd.merge(df, dfp[COLS + ['optPrice']], on=COLS)

In [ ]:
MINEXPROM = _vars.MINEXPROM
PREC = _vars.PREC
MINOPTSELLPRICE = _vars.MINOPTSELLPRICE

# make zero margin as nan
zero_margin_condition = df_nakeds.margin == 0
df_nakeds.loc[zero_margin_condition, 'margin'] = np.nan

# remove nans from margins
df_nakeds = df_nakeds[~df_nakeds.margin.isnull()].reset_index(drop=True)

# fill missing commissions with max per symbol
commissions = df_nakeds.groupby('symbol').comm.max().to_dict()
df_nakeds.comm = df_nakeds.comm.fillna(df_nakeds.symbol.map(commissions))

# fill remaining commissions
df_nakeds.comm = df_nakeds.comm.fillna(max(commissions.values()))

# fill margins
mgn_dict = df_nakeds.groupby('symbol').margin.max().to_dict()
cond = df_nakeds.margin.isnull()
df_nakeds.loc[cond, 'margin'] = df_nakeds[cond].symbol.map(mgn_dict)

# Get precise expected prices
xp = ((MINEXPROM*df_nakeds.dte/365*df_nakeds.margin) +
        df_nakeds.comm)/df_nakeds.lot/df_nakeds.multiplier

# Set the minimum option selling price
xp[xp < MINOPTSELLPRICE] = MINOPTSELLPRICE

# Make the expected Price
expPrice = pd.concat([xp, 
                        df_nakeds.optPrice], axis=1)\
                        .max(axis=1)
expPrice = expPrice.apply(lambda x: get_prec(x, PREC))

expPrice = expPrice + PREC # bump up the expected price a bit

dfn = df_nakeds.assign(expPrice = expPrice)

# Limit the number of options to trade
cond = dfn.expPrice < 3* dfn.optPrice
dfn = dfn[cond]



In [ ]:
dfn.iloc[(dfn.expPrice/(dfn.margin+dfn.comm)).argsort()[::-1]]